In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import shapely
import math

In [2]:
import os
#had to add GDAL_DATA variable to system variables and set value to the folder of gdal in C:\Users\mishaun\AppData\Local\Continuum\anaconda3\Library\share\gdal on my work computer
'GDAL_DATA' in os.environ

True

In [3]:
#These 2 lines of code will allow for all output to be displayed within a given cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reading in Data

In [4]:
shapezipfile = ("zip://Data/BLMWY-2020-Q1-3_WGS84.zip")
tractshp = gp.read_file(shapezipfile, encoding = "utf-8")

In [5]:
#this is shapefile downloaded from drillinginfo holding well information
prodshp = gp.read_file("zip://Data/production.ZIP")
permitshp = gp.read_file("zip://Data/permits.ZIP")

#reading in csv of leases - converting to a GeoDataFrame - initial coord system is epsg:4326
leases = pd.read_csv("Data/LeasesTable.CSV")
leasesgeo = gp.GeoDataFrame(leases, crs = {'init': 'epsg:4326'}, geometry=gp.points_from_xy(leases["Longitude (WGS84)"], leases["Latitude (WGS84)"]))

### Trimming and Cleaning Data

In [6]:
leasesgeo.columns

Index(['State/Province', 'Effective Date', 'Record Date',
       'Expiration of Primary Term', 'Term (Months)', 'Grantor',
       'Grantee Alias', 'Royalty', 'Bonus', 'Area (Acres)', 'Section',
       'Township', 'Township Direction', 'Range', 'Range Direction',
       'Vol/Page', 'Record Number', 'Instrument Type', 'Instrument Date',
       'County/Parish', 'Options/Extensions', 'DI Basin', 'Ext. Bonus',
       'Ext. Term (Months)', 'Abstract', 'Block', 'BLM', 'State Lease',
       'Grantee', 'Grantor Address', 'Grantee Address', 'Max Depth',
       'Majority Legal Assignee', 'DI Subplay', 'Min Depth',
       'Majority Assignment Effective Date', 'Latitude (WGS84)', 'DI Play',
       'Majority Legal Assignee Interest', 'Longitude (WGS84)',
       'Majority Assignment Vol/Page', 'geometry'],
      dtype='object')

In [7]:
leasesgeo.drop(columns = ['Instrument Type', 'Instrument Date','Options/Extensions', 'DI Basin', 'Ext. Bonus',
       'Ext. Term (Months)', 'Abstract', 'Block', 'BLM', 'State Lease',
       'Grantee', 'Grantor Address', 'Grantee Address', 'Max Depth',
       'Majority Legal Assignee', 'DI Subplay', 'Min Depth',
       'Majority Assignment Effective Date','Majority Legal Assignee Interest','Majority Assignment Vol/Page'], inplace = True)

In [8]:
prodshp.columns

Index(['APIUWI', 'OpAlias', 'LeaseName', 'WellNo', 'County', 'Reservoir',
       'ProdType', 'ProdStatus', 'DrillType', 'TD', 'SpudDate', 'FstPrdDate',
       'LstPrdDate', 'MoProd', 'CumGas', 'DailyGas', 'CumLiq', 'DailyLiq',
       'LatestLiq', 'LatestGas', 'CumWtr', 'CumBOE', 'DISubplay', '1moLiq',
       '1moGas', '6moLiq', 'DIBasin', '6moGas', '6moBOE', '6moWater', 'DIPlay',
       'PracIP_Liq', 'PracIP_BOE', 'PracIP_Gas', 'PrcIPCFGED', 'LatestWtr',
       'Prior12Liq', 'Prior12Gas', 'LastTestDt', 'Prior12Wtr', 'LastFlwPrs',
       'LastWHSIP', '2moGOR', 'LatestGOR', 'CumGOR', 'Lst12Yield', '2moYield',
       'LatestYld', 'PeakGas', 'PkGasMoNo', 'PeakLiq', 'PkLiqMoNo', 'PeakBOE',
       'PkBOEMoNo', 'PkMMCFGE', 'PkMMCFGMoN', 'TopPerf', 'BtmPerf', 'GasGrav',
       'OilGrav', 'CompDate', 'WellCount', 'MaxActvWel', 'GasGather',
       'LiqGather', 'LeaseNo', 'PerfLength', 'TVD', 'Field', 'State',
       'District', 'GeoProvin', 'Section', 'Country', 'Township', 'Range',
       'Lati

In [9]:
prodshp.drop(columns = ['LatestWtr','CumWtr',
       'Prior12Liq', 'Prior12Gas', 'LastTestDt', 'Prior12Wtr', 'LastFlwPrs',
       'LastWHSIP', '2moGOR', 'LatestGOR', 'CumGOR', 'Lst12Yield', '2moYield',
       'LatestYld', 'PeakGas', 'PkGasMoNo', 'PeakLiq', 'PkLiqMoNo', 'PeakBOE',
       'PkBOEMoNo', 'PkMMCFGE', 'PkMMCFGMoN', 'TopPerf', 'BtmPerf', 'GasGrav',
       'OilGrav','CompDate', 'GasGather',
       'LiqGather', 'LeaseNo'], inplace = True)

In [10]:
permitshp.columns

Index(['API10UWI', 'District', 'FiledDate', 'AprvdDate', 'ExpDate', 'State',
       'County', 'OpAlias', 'LeaseName', 'WellNo', 'Formation', 'PermDepth',
       'TVD', 'PermitType', 'WellType', 'DrillType', 'WellStatus',
       'PermStatus', 'Field', 'OpReported', 'AmendDate', 'CntctName',
       'CntctPhone', 'OperAddrs', 'OperCity', 'OperState', 'OperZip',
       'OperCity30', 'Section', 'OperCity50', 'Township', 'Range', 'Block',
       'Survey', 'TVD_UOM', 'Abstract', 'WGID', 'H2S_Area', 'Latitude',
       'Longitude', 'OFS_Reg', 'Btm_Lat', 'Btm_Lon', 'LeaseNo', 'PermDUOM',
       'PermitNo', 'DIBasin', 'DIPlay', 'DISubplay', 'OpCompany', 'OpTicker',
       'geometry'],
      dtype='object')

In [11]:
permitshp.drop(columns = ['OpReported', 'AmendDate', 'CntctName',
       'CntctPhone', 'OperAddrs', 'OperCity', 'OperState', 'OperZip',
       'OperCity30', 'Section', 'OperCity50', 'Township', 'Range', 'Block',
       'Survey', 'TVD_UOM', 'Abstract', 'WGID', 'H2S_Area','OFS_Reg', 'LeaseNo', 'PermDUOM',
       'PermitNo','OpCompany', 'OpTicker'], inplace=True)

In [12]:
leasesgeo["Record Date"] = pd.to_datetime(leasesgeo["Record Date"])
prodshp["FstPrdDate"] = pd.to_datetime(prodshp["FstPrdDate"])
permitshp["AprvdDate"] = pd.to_datetime(permitshp["AprvdDate"])

In [13]:
leasesgeo.info()
prodshp.info()
permitshp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 53266 entries, 0 to 53265
Data columns (total 22 columns):
State/Province                53266 non-null object
Effective Date                31049 non-null object
Record Date                   53266 non-null datetime64[ns]
Expiration of Primary Term    53266 non-null object
Term (Months)                 53266 non-null int64
Grantor                       53266 non-null object
Grantee Alias                 53239 non-null object
Royalty                       33903 non-null float64
Bonus                         18347 non-null float64
Area (Acres)                  52506 non-null float64
Section                       53266 non-null float64
Township                      53266 non-null float64
Township Direction            53266 non-null object
Range                         53266 non-null float64
Range Direction               53266 non-null object
Vol/Page                      53266 non-null object
Record Number                 53266 no

### Converting geodataframes to same coord system - UTM system for creating buffers

In [14]:
prodshp.to_crs(epsg = 26913, inplace = True)
permitshp.to_crs(epsg = 26913, inplace = True)
leasesgeo.to_crs(epsg = 26913, inplace = True)
tractshp.to_crs(epsg = 26913, inplace = True)

In [15]:
prodshp.crs
permitshp.crs
leasesgeo.crs

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

### Adding a column to tract shapefile data for centroids of each tract for creating buffers upon

In [16]:
tractshp["centroids"] = tractshp.centroid
#calculating acres of tract, conv to acres is m^2 to acres
tractshp['Acres'] = round(tractshp.area * 0.000247105)
tractshp.head()

,SaleParcel,lot_no,tract_id,short_code,label,geometry,centroids,Acres
0,WY-201Q-105,65012,2,BLMWY-2020-Q1-3,WY-2020-03-0374,"MULTIPOLYGON (((173527.819 4984155.461, 173534...",POINT (172183.913 4984182.822),941.0
1,WY-201Q-063,64970,16,BLMWY-2020-Q1-3,WY-2020-03-6207,"MULTIPOLYGON (((219007.722 4650891.363, 218687...",POINT (222418.619 4647579.181),1322.0
2,WY-201Q-001,64908,93,BLMWY-2020-Q1-3,WY-2020-03-6613,"POLYGON ((562927.151 4797521.250, 563330.976 4...",POINT (563130.227 4797125.862),80.0
3,WY-201Q-002,64909,102,BLMWY-2020-Q1-3,WY-2020-03-6660,"POLYGON ((514894.040 4607634.757, 515295.946 4...",POINT (515096.374 4607429.194),40.0
4,WY-201Q-003,64910,89,BLMWY-2020-Q1-3,WY-2020-03-6585,"MULTIPOLYGON (((518971.985 4775132.462, 519376...",POINT (520545.083 4773447.285),2316.0


In [17]:
#adding buffer around centroid point from tract of 3 mi (1609.34 meters = 1 mile)
milesbuffer = 3 * 1609.34

tractshp["buffers"] = tractshp.centroids.apply(lambda x: x.buffer(milesbuffer,20))


# Testing Spatial Filters

In [69]:
TestT = 40
tractTest = tractshp[tractshp["tract_id"] == TestT].iloc[0]


permFiltered = permitshp.within(tractshp[tractshp["tract_id"] ==TestT]["buffers"].iloc[0])
prodFiltered = prodshp.within(tractshp[tractshp["tract_id"] ==TestT]["buffers"].iloc[0])
leasesFiltered = leasesgeo.within(tractshp[tractshp["tract_id"]==TestT]["buffers"].iloc[0])

In [70]:
permFiltered.value_counts()

False    46457
True        12
dtype: int64

In [71]:
permitstoeval = permitshp.loc[permFiltered]

In [72]:
prodtoeval = prodshp.loc[prodFiltered]

In [73]:
prodFiltered.value_counts()

False    62874
True         6
dtype: int64

In [74]:
leasesFiltered.value_counts()

False    53264
True         2
dtype: int64

In [75]:
leasestoeval = leasesgeo.loc[leasesFiltered]

In [76]:
leasestoeval.drop_duplicates("Record Number",inplace = True)
leasestoeval

C:\Users\mishaun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,State/Province,Effective Date,Record Date,Expiration of Primary Term,Term (Months),Grantor,Grantee Alias,Royalty,Bonus,Area (Acres),...,Township Direction,Range,Range Direction,Vol/Page,Record Number,County/Parish,Latitude (WGS84),DI Play,Longitude (WGS84),geometry
4777,WY,NaN,2019-09-13,2020-08-23,12,PELLATZ ROBERT D ET AL,DEVON ENERGY,NaN,NaN,0.0,...,N,71.0,W,"=""01683/0691""",1087208,CONVERSE (WY),43.167017,POWDER RIVER,-105.389570,POINT (468332.890 4779435.622)
49813,WY,NaN,2014-08-05,2024-08-05,120,BLM,MBI O&G,0.125,3100.0,320.0,...,N,71.0,W,WY-1408-026/NA,WYW183606,CONVERSE (WY),43.239942,POWDER RIVER,-105.388968,POINT (468419.520 4787533.967)


### Calculating distance between leases within 3 mi radius and tract's centroid

In [77]:
#calculating distance (in miles) away lease within 3 mi radius is to tract of interest
leasestoeval["distance"] = leasestoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
leasestoeval

C:\Users\mishaun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,State/Province,Effective Date,Record Date,Expiration of Primary Term,Term (Months),Grantor,Grantee Alias,Royalty,Bonus,Area (Acres),...,Range,Range Direction,Vol/Page,Record Number,County/Parish,Latitude (WGS84),DI Play,Longitude (WGS84),geometry,distance
4777,WY,NaN,2019-09-13,2020-08-23,12,PELLATZ ROBERT D ET AL,DEVON ENERGY,NaN,NaN,0.0,...,71.0,W,"=""01683/0691""",1087208,CONVERSE (WY),43.167017,POWDER RIVER,-105.389570,POINT (468332.890 4779435.622),2.920035
49813,WY,NaN,2014-08-05,2024-08-05,120,BLM,MBI O&G,0.125,3100.0,320.0,...,71.0,W,WY-1408-026/NA,WYW183606,CONVERSE (WY),43.239942,POWDER RIVER,-105.388968,POINT (468419.520 4787533.967),2.483696


#test to get compass direction

a = leasestoeval["geometry"].iloc[0].x
b = leasestoeval["geometry"].iloc[0].y
c = tractTest["centroids"].x
d = tractTest["centroids"].y

#south and west are positive direction for my axis convention
western = c - a
southern = d - b

diff = western/southern
compdeg = math.atan(diff)
compdeg *= 57.2958
compdeg

if southern > 0:
    if compdeg < 25:
        carddir = "S"
    elif compdeg > 75:
        if western>0:
            carddir = "W"
        else:
            carddir = "E"
    else:
        if western>0:
            carddir = "SW"
        else:
            carddir = "SE"
else:
    if compdeg < 25:
        carddir = "S"
    elif compdeg > 75:
        if western>0:
            carddir = "W"
        else:
            carddir = "E"
    else:
        if western>0:
            carddir = "SW"
        else:
            carddir = "SE"  
                
carddir    
    

In [78]:
#function to get compass direction

def cardDir(point, tractRef):
    
    a = point.x
    b = point.y
    c = tractRef["centroids"].x
    d = tractRef["centroids"].y

    #south and west are positive direction for my axis convention
    western = c - a
    southern = d - b

    diff = western/southern
    compdeg = math.atan(diff)
    compdeg *= 57.2958
    compdeg

    if southern > 0:
        if compdeg < 25:
            carddir = "S"
        elif compdeg > 75:
            if western>0:
                carddir = "W"
            else:
                carddir = "E"
        else:
            if western>0:
                carddir = "SW"
            else:
                carddir = "SE"
    else:
        if compdeg < 25:
            carddir = "N"
        elif compdeg > 75:
            if western>0:
                carddir = "W"
            else:
                carddir = "E"
        else:
            if western>0:
                carddir = "NW"
            else:
                carddir = "NE"  
     
    return carddir           
   
    

In [79]:
leasestoeval["direction"] = leasestoeval["geometry"].apply(lambda x: cardDir(x, tractTest))

C:\Users\mishaun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
leasestoeval

,State/Province,Effective Date,Record Date,Expiration of Primary Term,Term (Months),Grantor,Grantee Alias,Royalty,Bonus,Area (Acres),...,Range Direction,Vol/Page,Record Number,County/Parish,Latitude (WGS84),DI Play,Longitude (WGS84),geometry,distance,direction
4777,WY,NaN,2019-09-13,2020-08-23,12,PELLATZ ROBERT D ET AL,DEVON ENERGY,NaN,NaN,0.0,...,W,"=""01683/0691""",1087208,CONVERSE (WY),43.167017,POWDER RIVER,-105.389570,POINT (468332.890 4779435.622),2.920035,S
49813,WY,NaN,2014-08-05,2024-08-05,120,BLM,MBI O&G,0.125,3100.0,320.0,...,W,WY-1408-026/NA,WYW183606,CONVERSE (WY),43.239942,POWDER RIVER,-105.388968,POINT (468419.520 4787533.967),2.483696,N


In [83]:

permitstoeval["distance"] = permitstoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
permitstoeval["direction"] = permitstoeval["geometry"].apply(lambda x: cardDir(x, tractTest))
permitstoeval

C:\Users\mishaun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mishaun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,API10UWI,District,FiledDate,AprvdDate,ExpDate,State,County,OpAlias,LeaseName,WellNo,...,Latitude,Longitude,Btm_Lat,Btm_Lon,DIBasin,DIPlay,DISubplay,geometry,distance,direction
16789,4900948392,None,None,2020-01-28,2022-01-27,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,02-113771-7XNH,...,43.217172,-105.338093,43.189325,-105.339596,POWDER RIVER,POWDER RIVER,None,POINT (472539.848 4784987.349),1.754175,NE
19790,4900948280,None,None,2020-01-28,2022-01-27,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,12-013771-5XNH,...,43.190962,-105.332950,43.217787,-105.320865,POWDER RIVER,POWDER RIVER,None,POINT (472945.938 4782074.913),2.157786,S
21385,4900943869,None,None,2019-01-17,2021-01-16,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,083771-2PH,...,43.188219,-105.413566,43.202753,-105.401556,POWDER RIVER,POWDER RIVER,None,POINT (466394.005 4781799.505),2.553509,SW
22427,4900942810,None,None,2018-12-27,2020-12-26,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,18-193771-5XNH,...,43.187166,-105.422393,43.160089,-105.427757,POWDER RIVER,POWDER RIVER,None,POINT (465676.091 4781686.159),2.979824,SW
22435,4900941533,None,None,2018-12-27,2020-12-26,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,09-043771-4XTH,...,43.187543,-105.389548,43.217245,-105.382894,POWDER RIVER,POWDER RIVER,None,POINT (468345.349 4781715.172),1.660044,SW
22524,4900943868,None,None,2019-01-17,2021-01-16,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,083771-1PH,...,43.188077,-105.413566,43.202741,-105.424659,POWDER RIVER,POWDER RIVER,None,POINT (466393.928 4781783.835),2.558411,SW
22959,4900948278,None,None,2020-01-28,2022-01-27,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,12-013771-1XNH,...,43.191156,-105.333660,43.217695,-105.334828,POWDER RIVER,POWDER RIVER,None,POINT (472888.357 4782096.743),2.120100,S
25458,4900942825,None,None,2018-12-27,2020-12-26,WY,CONVERSE (WY),DEVON,SDU TILLARD,18-193771-7XNH,...,43.187181,-105.422210,43.160116,-105.422813,POWDER RIVER,POWDER RIVER,None,POINT (465690.986 4781687.783),2.971106,SW
27787,4900948279,None,None,2020-01-28,2022-01-27,WY,CONVERSE (WY),DEVON,SDU TILLARD FED,12-013771-3XNH,...,43.191057,-105.333309,43.217745,-105.327488,POWDER RIVER,POWDER RIVER,None,POINT (472916.841 4782085.612),2.138845,S
40327,4900941632,None,None,2018-12-27,2020-12-26,WY,CONVERSE (WY),DEVON,SDU WYO Fed,16-213771-4XTH,...,43.187330,-105.389067,43.159891,-105.383016,POWDER RIVER,POWDER RIVER,None,POINT (468384.300 4781691.269),1.657512,SW


### Statistical summaries: bonus by year, closest lease, etc

In [32]:
leasestoeval.groupby(leasestoeval["Record Date"].apply(lambda x: x.year)).describe()

Term (Months)                                                      \
                    count   mean        std   min    25%    50%    75%    max   
Record Date                                                                     
2018                  6.0  110.0  24.494897  60.0  120.0  120.0  120.0  120.0   

            Royalty           ... Longitude (WGS84)             distance  \
              count     mean  ...               75%         max    count   
Record Date                   ...                                          
2018            6.0  0.13195  ...       -108.751921 -108.742085      6.0   

                                                                         \
                 mean       std       min       25%       50%       75%   
Record Date                                                               
2018         2.117768  0.460778  1.429576  1.976305  2.104985  2.244414   

                       
                  max  
Record Date            
2018         2.842761  

[1 rows x 80 columns]

In [33]:
leasestoeval["Record Date"]
leasestoeval.describe()["Bonus"]["mean"]

23111   2018-09-18
23116   2018-09-18
23270   2018-09-18
23278   2018-09-18
23404   2018-09-18
26898   2018-07-18
Name: Record Date, dtype: datetime64[ns]

5.0